In [1]:
import os

## 1. 選擇使用的GPU 

In [2]:
# manually select one or several free gpu
os.environ['CUDA_VISIBLE_DEVICES'] = '0, 1'
# use CPU only
# os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

## 2. 到當前資料夾
- cd C:/Users/Garin/Desktop/學長畢業資料/實驗程式yu/

## 3. 同步程式
### - local -> server
- scp -r -P 2222 ./config garin@140.113.214.40:/home/garin/Documents/depth/
- scp -r -P 2222 ./src garin@140.113.214.40:/home/garin/Documents/depth/

### - server -> local 
- scp -r -P 2222 garin@140.113.214.40:/home/garin/Documents/depth/config .
- scp -r -P 2222 garin@140.113.214.40:/home/garin/Documents/depth/src .

## 4. 建dataset

In [ ]:
%run data_preparation/kitti_data_prepare.py \
--dataset_dir=/work/garin0115/datasets/KITTI/ \
--dataset_name=kitti_raw_eigen \
--dump_root=/home/garin0115/datasets/kitti_3frames_256_832/ \
--seq_length=3 \
--img_height=256 \
--img_width=832 \
--num_threads=32 \
--remove_static

## 5. Train flow 

In [ ]:
%run ./main.py -c ../config/flow3.ini -t train_flow --cont_model=../results/KITTI_RAW_128_416_UnDepthflow_flow_pwc_b8_3frames/checkpoints/kitti_3frames/model-170987

## 6. Train depth & pose 

In [3]:
%run ./main.py -c ../config/dp3.ini -t train_dp \
--restore_flow_model=../results/KITTI_256_832_flow_b8_3frames/model-392302

ModuleNotFoundError: No module named 'tensorflow.contrib'

#### continue training depth and pose 

In [ ]:
%run ./main.py -c ../config/dp3.ini -t train_dp \
--cont_model=../results/KITTI_256_832_dp_b8_ShuffleNetV2_separable_share/checkpoints/kitti_3frames/model-15076 \
--restore_flow_model=../results/KITTI_256_832_flow_b8_3frames/model-392302

## 7. Test

In [ ]:
%run ./main.py -c ../config/test_dp_kitti.ini -t kitti_eval \
--restore_dp_model=../results/KITTI_256_832_dp_b8_ShuffleNetV2_separable_sep/checkpoints/kitti_3frames/model-35176
# --restore_dp_model=../results/KITTI_RAW_128_416_UnDepthflow_dp_b4_resnet50_3frames/checkpoints/kitti_3frames/model-342007


In [ ]:
%run kitti_eval/eval_depth.py --split=eigen --kitti_dir=/work/garin0115/datasets/KITTI/ \
--pred_file=../results/kitti/test_kitti.npy \
--depth_results=../results/kitti_depths

#### 將結果傳回本地端 

scp -r -P 2222 garin@140.113.214.40:/home/garin/Documents/depth/results/kitti_depths/ C:/Users/Garin/Desktop/學長畢業資料/實驗程式yu/result

## TensorBoard 

In [ ]:
!tensorboard --logdir=../results/KITTI_256_832_dp_b8_ShuffleNetV2_separable_share/ \
--samples_per_plugin images=100